#### Importação das bibliotecas necessárias

Nesta célula, são importadas as bibliotecas essenciais para manipulação de dados e geração de valores aleatórios:
- `pandas` para manipulação de tabelas e DataFrames,
- `numpy` para operações numéricas,
- `random` para sorteios e geração de números aleatórios.

In [ ]:
import pandas as pd
import numpy as np
import random

#### Leitura dos dados e preparação dos DataFrames

Aqui, os arquivos de alimentos e nutrientes são lidos. Em seguida:
- O código do alimento recebe o prefixo 'BR'.
- Os DataFrames são mesclados para unir as informações.
- É criada uma coluna de quantidade aleatória para cada alimento.
- Inicializa-se o valor energético total como zero.
- Define-se um dicionário para separar os alimentos em grupos alimentares.

In [ ]:
alimentos = pd.read_excel('alimentos.xlsx')
nutrientes = pd.read_excel('alimento_nut_all.xlsx')

nutrientes['cod_alimento'] = nutrientes['cod_alimento'].apply(lambda x: 'BR' + x)

alimentos = pd.merge(alimentos, nutrientes, on='cod_alimento', how='inner')
alimentos['quantidade'] = random.uniform(0.1, 1.5)
alimentos['valor_energetico_total'] = 0

grupos_alimentares = {
   'proteina': [69, 70, 74, 78],
   'vegetal': [66, 67, 84, 85],
   'carboidrato': [75, 76, 77, 82],
   'bebida': [72]
}

#### Separação dos alimentos por grupos

Esta célula filtra o DataFrame de alimentos em quatro grupos principais: proteína, vegetal, carboidrato e bebida, conforme os IDs definidos anteriormente.

In [ ]:
grupo_proteina = alimentos[alimentos['grupo_id'].isin(grupos_alimentares['proteina'])]
grupo_vegetal = alimentos[alimentos['grupo_id'].isin(grupos_alimentares['vegetal'])]
grupo_carboidrato = alimentos[alimentos['grupo_id'].isin(grupos_alimentares['carboidrato'])]
grupo_bebida = alimentos[alimentos['grupo_id'].isin(grupos_alimentares['bebida'])]

#### Função para calcular o valor energético total de uma refeição

Define uma função que recebe um DataFrame de refeição e calcula o valor energético total multiplicando a energia média de cada alimento pela sua quantidade.

In [ ]:
def calcular_valor_energetico(refeicao):
    # Calcular a quantidade total e o valor energético total para a refeição
    valor_energetico_total = 0
    for i in range(refeicao.shape[0]):
        valor_energetico_total += refeicao.iloc[i]['energia1_media'] * refeicao.iloc[i]['quantidade']
    return valor_energetico_total

#### Geração de casos iniciais de refeições

Esta função cria 50 refeições aleatórias, selecionando alimentos dos diferentes grupos, atribuindo quantidades aleatórias e calculando o valor energético total de cada refeição.

In [ ]:
def gerar_casos_iniciais():
    casos = []

    for _ in range(50):
        refeicao = pd.DataFrame(columns=alimentos.columns)
        quant = 0

        # Selecionar aleatoriamente um alimento de proteína
        alimento_proteina = random.choice(grupo_proteina['cod_alimento'].values.tolist())
        refeicao = pd.concat([refeicao, alimentos[alimentos['cod_alimento'] == alimento_proteina]])
        
        # Selecionar aleatoriamente dois ou mais vegetais
        alimentos_vegetais = random.sample(grupo_vegetal['cod_alimento'].tolist(), random.randint(2, 4))
        refeicao = pd.concat([refeicao, alimentos[alimentos['cod_alimento'].isin(alimentos_vegetais)]])
        
        # Selecionar aleatoriamente um ou mais carboidratos
        alimentos_carboidratos = random.sample(grupo_carboidrato['cod_alimento'].tolist(), random.randint(1, 2))
        refeicao = pd.concat([refeicao, alimentos[alimentos['cod_alimento'].isin(alimentos_carboidratos)]])
        
        # Selecionar aleatoriamente uma bebida
        alimento_bebida = random.choice(grupo_bebida['cod_alimento'].tolist())
        refeicao = pd.concat([refeicao, alimentos[alimentos['cod_alimento'] == alimento_bebida]])

        quant = 2 + alimentos_vegetais.__len__() + alimentos_carboidratos.__len__()
        refeicao['quantidade'] = [random.uniform(0.1, 1.5) for _ in range(quant)]
        valor_energetico_total = calcular_valor_energetico(refeicao)  

        # Adicionar quantidade total e valor energético total à refeição
        refeicao['valor_energetico_total'] = valor_energetico_total
                
        casos.append(refeicao.reset_index(drop=True))
    return casos

#### Seleção do caso mais próximo do valor energético desejado

Gera os casos iniciais, solicita ao usuário o valor energético médio e o limiar de tolerância, e identifica qual refeição gerada está mais próxima do valor informado.

In [ ]:
base_de_casos = gerar_casos_iniciais()

energia1_med = float(input("Insira o valor médio de energia (em kj): "))
limiar = float(input("Insira o limiar: "))

diferencas = []
index = []
i = 0
dicionario = {}

for caso in base_de_casos:
    dif = abs(caso['valor_energetico_total'].values[0] - energia1_med)
    diferencas.append(dif)
    index.append(i)
    dicionario[i] = dif
    i += 1

dicionario = dict(sorted(dicionario.items(), key=lambda item: item[1]))

caso_mais_proximo = base_de_casos[list(dicionario.keys())[0]].copy()

#### Adaptação da refeição para se aproximar do valor energético desejado

Verifica se o valor energético da refeição está dentro do intervalo aceitável. Se não estiver, ajusta as quantidades dos alimentos proporcionalmente para tentar atingir o valor desejado.

In [ ]:
def adaptar_refeicao(caso_mais_proximo, energia1_med, limiar):
    if caso_mais_proximo['valor_energetico_total'].values[0] < energia1_med + limiar and caso_mais_proximo['valor_energetico_total'].values[0] > energia1_med - limiar:
        print("Caso aceito")
        print(caso_mais_proximo)
    else:
        caso_mais_proximo.reset_index(drop=True, inplace=True)
        caso_mais_proximo.index += 1

        print("Caso rejeitado. Fazendo ajustes...")
        porc = energia1_med / caso_mais_proximo['valor_energetico_total'].values[0]

        for i in range(caso_mais_proximo.shape[0]):
            caso_mais_proximo['quantidade'].iloc[i] *= porc

        caso_mais_proximo['valor_energetico_total'] = calcular_valor_energetico(caso_mais_proximo)

adaptar_refeicao(caso_mais_proximo, energia1_med, limiar)
print(caso_mais_proximo)

#### Avaliação da refeição pelo usuário e possibilidade de ajuste

Permite ao usuário aceitar ou rejeitar a refeição sugerida. Caso rejeite, pode substituir um alimento por outro do mesmo grupo, recalculando o valor energético e ajustando novamente se necessário.

In [ ]:
avaliacao_usuario = 0

while avaliacao_usuario != 1:
    avaliacao_usuario = input("A refeicao gerada, é atrativa? (0/1): ")

    if avaliacao_usuario == '1':
        print("Refeição aceita na base de casos")
        base_de_casos.append(caso_mais_proximo)
        break
    else:
        print("Refeição rejeitada na base de casos")

        ingrediente = str(input("Insira o código do alimento que deseja alterar (exemplo: BRC0001B): "))
        print(caso_mais_proximo['cod_alimento'].values.tolist())
        if ingrediente not in (caso_mais_proximo['cod_alimento'].values).tolist():
            print("Alimento não encontrado")
            break
        alimento = caso_mais_proximo[caso_mais_proximo['cod_alimento'] == ingrediente]
        grupo = alimento[alimento['cod_alimento'] == ingrediente]['grupo_id'].values[0]

        index_alimento = caso_mais_proximo[caso_mais_proximo['cod_alimento'] == ingrediente].index[0]
        caso_mais_proximo = caso_mais_proximo.drop(index_alimento)

        if grupo in grupos_alimentares['proteina']:
            alimento_proteina = pd.DataFrame(columns=caso_mais_proximo.columns)
            alimento_proteina = random.choice(grupo_proteina['cod_alimento'].values.tolist())
            caso_mais_proximo = pd.concat([caso_mais_proximo, alimentos[alimentos['cod_alimento'] == alimento_proteina]])
        if grupo in grupos_alimentares['vegetal']:
            alimento_vegetal = pd.DataFrame(columns=caso_mais_proximo.columns)
            alimento_vegetal = random.choice(grupo_vegetal['cod_alimento'].values.tolist())
            caso_mais_proximo = pd.concat([caso_mais_proximo, alimentos[alimentos['cod_alimento'] == alimento_vegetal]])
        if grupo in grupos_alimentares['carboidrato']:
            alimento_carboidrato = pd.DataFrame(columns=caso_mais_proximo.columns)
            alimento_carboidrato = random.choice(grupo_carboidrato['cod_alimento'].values.tolist())
            caso_mais_proximo = pd.concat([caso_mais_proximo, alimentos[alimentos['cod_alimento'] == alimento_carboidrato]])
        if grupo in grupos_alimentares['bebida']:
            alimento_bebida = pd.DataFrame(columns=caso_mais_proximo.columns)
            alimento_bebida = random.choice(grupo_bebida['cod_alimento'].values.tolist())
            caso_mais_proximo = pd.concat([caso_mais_proximo, alimentos[alimentos['cod_alimento'] == alimento_bebida]])
        
        caso_mais_proximo['valor_energetico_total'] = calcular_valor_energetico(caso_mais_proximo)
        adaptar_refeicao(caso_mais_proximo, energia1_med, limiar)
        print("Nova refeição:")
        print(caso_mais_proximo)

#### Exibição dos alimentos e nutrientes do plano alimentar

Calcula e exibe a energia total (convertida para outra unidade), além dos principais nutrientes da refeição final. Mostra também a lista de alimentos e suas quantidades.

In [ ]:
energia = caso_mais_proximo['valor_energetico_total'].values[0]
energia = energia * 0.239005736
carboidratos = 0
proteinas = 0
lipidios = 0
fibras = 0
vit_c = 0

alimento = pd.DataFrame(columns= 'Código Alimento Qtde Valor'.split())

for i in range(caso_mais_proximo.shape[0]):

    alimento = pd.concat([alimento, pd.DataFrame({'Código': [caso_mais_proximo.iloc[i]['cod_alimento']], 'Alimento': [caso_mais_proximo.iloc[i]['nome_pt_br']], 'Qtde': [caso_mais_proximo.iloc[i]['quantidade']], 'Valor': [caso_mais_proximo.iloc[i]['energia2_media'] * caso_mais_proximo.iloc[i]['quantidade']]})])
   
    carboidratos += caso_mais_proximo.iloc[i]['carboidrato_total_media'] * caso_mais_proximo.iloc[i]['quantidade']
    proteinas += caso_mais_proximo.iloc[i]['proteina_media'] * caso_mais_proximo.iloc[i]['quantidade']
    lipidios += caso_mais_proximo.iloc[i]['lipidios_media'] * caso_mais_proximo.iloc[i]['quantidade']
    fibras += caso_mais_proximo.iloc[i]['fibra_alimentar_media'] * caso_mais_proximo.iloc[i]['quantidade']
    vit_c += caso_mais_proximo.iloc[i]['vitamina_C_media'] * caso_mais_proximo.iloc[i]['quantidade']

data = {
    "Nutriente": ["Energia", "Carboidratos", "Proteínas", "Lipídios", "Fibras", "Vitamina C"],
    "Valor e Unidade": [f"{energia:.2f} {caso_mais_proximo['energia2_unidade'].iloc[0]}",
                        f"{carboidratos:.2f} {caso_mais_proximo['carboidrato_total_unidade'].iloc[0]}",
                        f"{proteinas:.2f} {caso_mais_proximo['proteina_unidade'].iloc[0]}",
                        f"{lipidios:.2f} {caso_mais_proximo['lipidios_unidade'].iloc[0]}",
                        f"{fibras:.2f} {caso_mais_proximo['fibra_alimentar_unidade'].iloc[0]}",
                        f"{vit_c:.2f} {caso_mais_proximo['vitamina_C_unidade'].iloc[0]}"]
}

print("Alimentos do plano alimentar")
print(alimento.to_string(index=False))
df = pd.DataFrame(data)
print("Nutrientes do plano alimentar")
print(df.to_string(index=False))